In [1]:
import os
import shutil

original_dataset_dir = './dataset'
# dataset안에 있는 하위 모든 폴더의 목록을 가져온다
classes_list = os.listdir(original_dataset_dir)

# 기본 경로
base_dir = './splitted'
os.mkdir(base_dir)

# train, val, test 용도의 사진을 저장하기 위해 폴더 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 하위 폴더 목록을 동일하게 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))    
    os.mkdir(os.path.join(test_dir, clss))    

In [2]:
import math
# 모든 하위 폴더에서 동작
for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    # 경로에 있는 모든 이미지 파일의 목록을 받아온다
    fnames = os.listdir(path)
    
    # train, val, test 용도의 비율을 조절한다
    # 6:2:2 사용
    train_size = math.floor(len(fnames)*0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)
    
    # train 데이터 만큼을 train에 저장
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ',len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src,dst)
        
    # val 데이터 만큼을 val에 저장
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ',len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src,dst)
    
    # test 데이터 만큼을 test에 저장
    test_fnames = fnames[(validation_size + train_size):(test_size + validation_size + train_size)]
    print('Test size(',clss,'): ',len( test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src,dst)

Train size( Apple___Apple_scab ):  600
Validation size( Apple___Apple_scab ):  200
Test size( Apple___Apple_scab ):  200
Train size( Apple___Black_rot ):  600
Validation size( Apple___Black_rot ):  200
Test size( Apple___Black_rot ):  200
Train size( Apple___Cedar_apple_rust ):  600
Validation size( Apple___Cedar_apple_rust ):  200
Test size( Apple___Cedar_apple_rust ):  200
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Background_without_leaves ):  685
Validation size( Background_without_leaves ):  228
Test size( Background_without_leaves ):  228
Train size( Blueberry___healthy ):  901
Validation size( Blueberry___healthy ):  300
Test size( Blueberry___healthy ):  300
Train size( Cherry___healthy ):  600
Validation size( Cherry___healthy ):  200
Test size( Cherry___healthy ):  200
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew )

In [3]:
import torch

# GPU 사용 체크
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print('Current cuda device is', DEVICE)

# batch_size, epoch 설정 
BATCH_SIZE = 128
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# compose는 이미지 데이터의 전처리, Augmentaion에 사용
# Augmentaion의 경우 좌우반전, 밝기 조절, 이미지 임의 확대 등
# 이미지는 64*64 로 설정
transform_base = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()])

# ImageFolder는 dataset을 불러오는 메서드 
train_dataset = ImageFolder(root='./splitted/train',
                            transform = transform_base)
val_dataset = ImageFolder(root='./splitted/val',
                            transform = transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE, shuffle = True,
                                           num_workers = 4)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)

Current cuda device is cuda


In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    
    # 모델 레이어 설정
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)
        
    # forward 설정
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
# 모델 객체 생성 및 optimizer 설정
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [5]:
print(model_base)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=33, bias=True)
)


In [6]:
# 모델 학습
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [7]:
def evaluate(model, test_loader):
    model.eval()
    # 미니 배치별 loss 합산
    test_loss = 0
    # 맞은 갯수 
    correct = 0
    
    # 파라미터 업데이트 중단
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # loss 계산 cross entropy 사용
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 예측값 추출
            pred = output.max(1, keepdim=True)[1]
            # view as 로 Tensor 재정렬 후 비교연산 시행
            correct += prd.eq(target.view_as(pred)).sum().item()
            
    # 미니 배치당 평균 loss 
    test_loss /= len(test_loader.dataset)
    # 미니 배치당 평균 정확도
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
import time
import copy

torch.backends.cudnn.enabled = False
def train_baseline(model, train_loader, val_loaderm,
                   optimizer, num_epochs = 30):
    # 최고 정확도 저장을 위한 변수
    best_acc = 0.0
    # 최고 정확도 모델을 저장하기 위한 변수
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        # 학습 Loss와 정확도
        train_loss, train_acc = evaluate(model, train_loader)
        # 검증 Loss와 정확도
        val_loss, val_acc = evaluate(model, val_loader)
        # 가장 좋은 경우에 최고 정확도, 모델 갱신
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        time_elapsed = time.time() - since
        print('-------------- epoch {} --------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    # 최고 정확도 모델을 불러오고 이를 return
    model.load_state_dict(best_model_wts)
    return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
# 학습완료 모델 저장
torch.save(base, 'baseline.pt')

RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`